In [1]:
from pyspark.sql import SparkSession

sc = SparkSession.builder.appName('decision trees').getOrCreate()

24/09/07 15:26:57 WARN Utils: Your hostname, DESKTOP-2J74AJH resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/07 15:26:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/07 15:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/07 15:26:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/07 15:26:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/07 15:26:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/09/07 15:26:58 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/09/07 15:26:58 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [2]:
data_without_header = sc.read.option('inferSchema', True).option('header', False).csv('covtype.csv')
data_without_header.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

In [3]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

colnames =  ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"]
colnames = colnames + [f'Wilderness_Area_{i}' for i in range(4)]
colnames = colnames + [f'Soil_Type{i}' for i in range(40)] + ['Cover_Type']

data = data_without_header.toDF(*colnames)
data = data.withColumn('Cover_Type', col('Cover_Type').cast(DoubleType()))

In [4]:
train, test = data.randomSplit([0.9, 0.1])

In [5]:
from pyspark.ml.feature import VectorAssembler

input_cols = colnames[: len(colnames) - 1]
vector_assembler = VectorAssembler(inputCols= input_cols, outputCol = 'featureVector')
assembled_train = vector_assembler.transform(train)

In [6]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(seed = 1234, labelCol= 'Cover_Type', featuresCol= 'featureVector', predictionCol= 'prediction')
model = classifier.fit(assembled_train)
print(model.toDebugString)

24/09/07 15:27:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DecisionTreeClassificationModel: uid=DecisionTreeClassifier_8f9577437c20, depth=5, numNodes=43, numClasses=8, numFeatures=54
  If (feature 0 <= 3046.5)
   If (feature 0 <= 2481.5)
    If (feature 3 <= 15.0)
     If (feature 12 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 4.0
      Else (feature 23 > 0.5)
       Predict: 3.0
     Else (feature 12 > 0.5)
      Predict: 6.0
    Else (feature 3 > 15.0)
     If (feature 16 <= 0.5)
      Predict: 3.0
     Else (feature 16 > 0.5)
      If (feature 9 <= 1305.5)
       Predict: 3.0
      Else (feature 9 > 1305.5)
       Predict: 4.0
   Else (feature 0 > 2481.5)
    If (feature 17 <= 0.5)
     If (feature 15 <= 0.5)
      Predict: 2.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2702.5)
      Predict: 3.0
     Else (feature 0 > 2702.5)
      If (feature 5 <= 1227.0)
       Predict: 5.0
      Else (feature 5 > 1227.0)
       Predict: 2.0
  Else (feature 0 > 3046.5)
   If (feature 0 <= 3

In [9]:
import pandas as pd

df = pd.DataFrame(model.featureImportances.toArray(), index= input_cols, columns= ['importance']).sort_values(by= 'importance', ascending= False)
print(df)

                                    importance
Elevation                             0.829831
Soil_Type3                            0.040651
Soil_Type1                            0.032267
Hillshade_Noon                        0.026839
Horizontal_Distance_To_Hydrology      0.019267
Soil_Type31                           0.018413
Wilderness_Area_2                     0.015613
Horizontal_Distance_To_Roadways       0.004735
Vertical_Distance_To_Hydrology        0.003614
Soil_Type2                            0.003604
Horizontal_Distance_To_Fire_Points    0.002298
Aspect                                0.001801
Soil_Type9                            0.001067
Slope                                 0.000000
Wilderness_Area_3                     0.000000
Wilderness_Area_1                     0.000000
Wilderness_Area_0                     0.000000
Soil_Type0                            0.000000
Soil_Type4                            0.000000
Soil_Type5                            0.000000
Hillshade_3pm